In [ ]:
# to use cd, etc
import sys
import os

import numpy as np
# for math work
from numpy import linalg, matlib
# save, import and load data
from numpy import asarray, save, load

# needed for some matrix manipulations
import cvxpy as cp
# import math 

import scipy
# for curve fitting
from scipy import optimize
from scipy.io import loadmat  # this is the SciPy module that loads mat-files

from sklearn.metrics import r2_score, mean_squared_error

# import mat73
import pandas as pd

import matplotlib.pyplot as plt
# for plots
import seaborn
# set up seaborn for the plots
seaborn.set()

# suppress deprecation warnings
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

# # # to save data plots
# from matplotlib.backends.backend_pdf import PdfPages

# from datetime import datetime
import pickle

In [ ]:
sessStart = '072312'
sessEnd = '080412'
savepath = '/Users/mmadduri/Documents/PhD/ResearchMaterials/Code/BMI_Model/data/' + 'jeev' + sessStart + '_' + sessEnd + '/'

In [ ]:
with open(savepath + "cursorData.pkl", 'rb') as infile:
     kinList = pickle.load(infile)
kinData = np.asarray(kinList)[0]

In [ ]:
with open(savepath + "decoder_2_neuralData.pkl", 'rb') as infile:
     decNum, usedUnits, succIdx, nData_succ = pickle.load(infile)

In [ ]:
nB = nData_succ.shape[0]
nN = nData_succ.shape[1]
nT = nData_succ.shape[2]

In [ ]:
usedUnits

In [ ]:
decNum

In [ ]:
succIdx

In [ ]:
kinSucc = kinData[succIdx]

In [ ]:
len(kinData[0])

In [ ]:
trLen = 20

kinLen = np.zeros(len(kinSucc))
overcount = 0
for iK in range(len(kinSucc)):
    kinLen[iK] = len(kinSucc[iK])    
    if (len(kinSucc[iK]) >= trLen):
        overcount = overcount + 1

In [ ]:
overcount

In [ ]:
for iK in range(100):
    plt.plot(kinSucc[iK][:trLen, 0], kinSucc[iK][:trLen, 1]);
plt.show()

In [ ]:
for iK in range(10):
    plt.plot(kinSucc[iK][:trLen, 0], kinSucc[iK][:trLen, 1]);
plt.show()

In [ ]:
(kinSucc[0]).flatten()

In [ ]:
#rescale to mean 0
def standardizeArray(array_in):
    return (array_in - np.mean(array_in))/ (np.max(array_in) - np.min(array_in))

In [ ]:
k_flat = []
nd_flat = []
tr = []
for iK in range(len(kinSucc)):
    if (len(kinSucc[iK]) >= trLen):
        tr.append(iK)
        # make the cursor matrix
        k_iter = kinSucc[iK][0:trLen, :]
        k_flat.append(k_iter.flatten())
        
        # make the neural activity matrix
        k0 = kinSucc[iK][0, :]
        nd_tr = nData_succ[0:trLen, :, iK].flatten()
#         np.concatenate((k0, nd_tr))
        nd_flat.append(np.concatenate((k0, nd_tr)))
        
        
k_flat = np.array(k_flat, dtype=float).T
nd_flat = np.array(nd_flat, dtype=float).T

In [ ]:
k_flat.shape

In [ ]:
nd_flat.shape

In [ ]:
dec_calc = k_flat@np.linalg.pinv(nd_flat)
dec_calc.shape

In [ ]:
np.linalg.pinv(nd_flat).shape

In [ ]:
def plot_r2(xdata, ydata):
    plt.scatter(xdata, ydata)
    slope, intercept, r, p, se = scipy.stats.linregress(np.ndarray.flatten(xdata), np.ndarray.flatten(ydata))
    
    plt.plot(np.linspace(min(np.sort(np.ndarray.flatten(xdata))),max(np.sort(np.ndarray.flatten(xdata))),10),np.linspace(min(np.sort(np.ndarray.flatten(xdata))),max(np.sort(np.ndarray.flatten(xdata))),10)*slope+ intercept,'r')
#     plt.xlabel("cursor " + str(label))
#     plt.ylabel("prediction: cursor " + str(label))
    plt.title("r-squared = " + str(r))
    plt.show()

In [ ]:
kin_calc = np.zeros((k_flat.shape[0] , len(tr)))
for iT_idx, iT_val in enumerate(tr):
    kin_calc[:, iT_idx] = dec_calc@nd_flat[:, iT_idx]

In [ ]:
plt.xlabel("Actual")
plt.ylabel("Calculated")
plot_r2(k_flat.flatten(),kin_calc.flatten())

In [ ]:
dec_calc.shape

In [ ]:
nd_flat[:,0].shape